# データ加工

[前回](001_データ収集.ipynb)で収集したデータを、解析しやすいように加工する。  
具体的には、
* 横浜Ｍ+横浜Ｆ→横浜FMのようなことがあるので、2018年現在残っているチームを基準に、データ名をそろえる
* チームだけでなくスタジアム名も、日本平→アイスタのように単なる名前変更があったりするので、この辺を綺麗にする
* 不要なデータの削除(放送局や入場者数はあまり勝敗と関係なさそうなのでカットしておく)
* ホームチームの勝ち・負け・引き分けの形でわかるように結果を加工
* 第〇節か、何月に実施されたかといった時期情報がわかるよう加工

## まず保存したデータを取得

[前回](001_データ収集.ipynb)作成したスクリプトより、data/jleague.dbが存在する状況から開始とする。

In [1]:
import sqlite3
import pandas as pd
DB_NAME = 'data/jleague.db'
conn = sqlite3.connect(DB_NAME)
df = pd.read_sql("select * from result", conn)
conn.close()

#display(df)

In [2]:
len(df)

16055

## 不要データの削除

* インターネット中継・TV放送
* 入場者数

のデータを削除する。

また、サテライトリーグのデータは公式戦とはいえ勝敗データとして使いにくいので、削除しておく。  
オールスターゲームも同様。

In [3]:
df["大会"].drop_duplicates()

0                      ＹＮＣ
48                Ｊ１ サントリー
138               Ｊ１ ＮＩＣＯＳ
267              チャンピオンシップ
269           ゼロックススーパーカップ
917                     Ｊ１
1037           ＹＮＣ 予選Ａグループ
1041           ＹＮＣ 予選Ｂグループ
1165          ＹＮＣ 決勝トーナメント
1272              サントリーカップ
1279           ＹＮＣ 予選Ｃグループ
1281           ＹＮＣ 予選Ｄグループ
1283           ＹＮＣ 予選Ｅグループ
1336                Ｊ１ １ｓｔ
1472                Ｊ１ ２ｎｄ
1974                  Ｊ１参入
2000                    Ｊ２
5203               Ｊ１・Ｊ２入替
10938               Ｊ１昇格ＰＯ
11776             Ｊ２・ＪＦＬ入替
11829                   Ｊ３
12802              Ｊ２・Ｊ３入替
13872        明治安田チャンピオンシップ
13977    ＹＮＣ グループステージＡグループ
13980    ＹＮＣ グループステージＢグループ
14653       ＹＬＣ ノックアウトステージ
15021    ＹＬＣ グループステージＡグループ
15024    ＹＬＣ グループステージＢグループ
15469        ＹＬＣ プレーオフステージ
Name: 大会, dtype: object

In [4]:
# 不要データを削除
df = df.drop("index",axis=1)
df = df.drop("インターネット中継・TV放送",axis=1)
df = df.drop("入場者数",axis=1)
df = df[df["大会"]!="サテライトリーグ"].reset_index(drop=True)
df = df[df["大会"]!="サテライトリーグ Ａグループ"].reset_index(drop=True)
df = df[df["大会"]!="サテライトリーグ Ｂグループ"].reset_index(drop=True)
df = df[df["大会"]!="サテライトリーグ Ｃグループ"].reset_index(drop=True)
df = df[df["大会"]!="サテライトリーグ Ｄグループ"].reset_index(drop=True)
df = df[df["大会"]!="サテライトリーグ Ｅグループ"].reset_index(drop=True)
df = df[df["大会"]!="サテライトリーグ Ｆグループ"].reset_index(drop=True)
df = df[df["大会"]!="オールスター"].reset_index(drop=True)
df = df[df["大会"]!="ドリームマッチ"].reset_index(drop=True)
df = df[df["大会"]!="ＪＯＭＯ　ＣＵＰ"].reset_index(drop=True)
df = df[df["大会"]!="スペシャルマッチ"].reset_index(drop=True)
df = df[df["大会"]!="明治安田ワールドチャレンジ"].reset_index(drop=True)

display(df)

KeyError: "['インターネット中継・TV放送'] not found in axis"

In [5]:
len(df)

16055

## チーム名の統一

吸収合併やホームタウン変更、表記の揺らぎによりチーム名が変わったものについて、新しいチーム名を基準に変更していく。
* Ｖ川崎→東京Ｖ(ホームタウン変更)
* 横浜Ｍ→横浜FM(横浜フリューゲルスを吸収)
* 市原→千葉(ホームタウン拡大)
* Ｆ東京→FC東京(表記の揺らぎ)
* 平塚→湘南(ホームタウン拡大)
* B仙台→仙台(ブランメル仙台→ベガルタ仙台と名称変更)
* 草津→群馬(ホームタウン拡大)

In [6]:
len(df["ホーム"].drop_duplicates())

65

In [7]:
def rename_team(df, old, new):
    df["ホーム"] = df["ホーム"].replace(old,new)
    df["アウェイ"] = df["アウェイ"].replace(old,new)
    return df

df = rename_team(df, "Ｖ川崎", "東京Ｖ")
df = rename_team(df, "横浜M", "横浜FM")
df = rename_team(df, "市原", "千葉")
df = rename_team(df, "Ｆ東京", "FC東京")
df = rename_team(df, "平塚", "湘南")
df = rename_team(df, "B仙台", "仙台")
df = rename_team(df, "草津", "群馬")

In [9]:
#display(df)
len(df["ホーム"].drop_duplicates())

58

## スタジアム名の統一

スタジアム名はネーミングライツの売却がクラブの収入になるため、横浜Ｆマリノスのホームである「日産スタジアム」のように企業名が入ることが多い。  
昔はスタジアム名は単に地名になるものが多かったが、後にネーミングライツビジネスが始まったため、変更されたスタジアム名は多く、しかも命名権には期限があるため突然元に戻ったり、違う企業名になったりすることがある。  
この辺もわかりづらいので、新しい方に合わせる形で入れ替えておく。

### 2018年現在ホームスタジアムとして使われているもの

|スタジアム                       |略称  |チーム|旧称・別称|
|-------------------------------|-----|---|------|
|札幌ドーム                       |札幌ド |札幌|別称として「札幌」？|
|札幌厚別公園競技場                |札幌厚別 |札幌||
|千代台公園陸上競技場               |函館|札幌||
|ユアテックスタジアム仙台           |ユアスタ|仙台  |別称として「仙台」？|
|宮城スタジアム                    |宮城ス|仙台    ||
|県立カシマサッカースタジアム        |カシマ|鹿島||
|埼玉スタジアム2002                |埼玉  |浦和  ||
|さいたま市駒場スタジアム           |駒場  |浦和  ||
|三協フロンテア柏スタジアム          |柏   |柏    ||
|柏の葉公園総合競技場               |柏    |柏の葉||
|味の素スタジアム                  |味スタ|FC東京|東京スタジアム(東京)|
|江東区夢の島陸上競技場             |夢の島|FC東京U-23||
|等々力陸上競技場                  |等々力|川崎Ｆ  ||
|日産スタジアム                    |日産ス|横浜FM||
|Shonan BMW スタジアム平塚         |ＢＭＷス|湘南  |平塚競技場(平塚)|
|IAIスタジアム日本平               |アイスタ|清水  |静岡市清水日本平運動公園球技場(日本平)→アウトソーシングスタジアム(アウスタ)|
|エコパスタジアム                  |エコパ|清水|静岡スタジアム(静岡)|
|ヤマハスタジアム（磐田）            |ヤマハ|磐田  |別名として「磐田」？|
|パロマ瑞穂スタジアム               |パロ瑞穂|名古屋|瑞穂公園陸上競技場(瑞穂陸)|
|豊田スタジアム                    |豊田ス  |名古屋||
|パナソニックスタジアム吹田          |吹田Ｓ|Ｇ大阪 ||
|万博記念競技場                    |万博|Ｇ大阪||
|キンチョウスタジアム               |金鳥スタ|Ｃ大阪 ||
|ヤンマースタジアム長居             |ヤンマー|Ｃ大阪|長居陸上競技場(長居)|
|ノエビアスタジアム神戸             |ノエスタ|神戸  |神戸市立中央球技場(神戸中央)→御崎公園球技場神戸ウイングスタジアム(神戸ウイ)→ホームズスタジアム(ホムスタ・ホームズ)|
|神戸総合運動公園ユニバー記念競技場    |神戸ユ|神戸||
|エディオンスタジアム広島            |Ｅスタ|広島  |広島ビッグアーチ(広島ビ)|
|広島県総合グランドメインスタジアム   |広島ス|広島  ||
|ベストアメニティスタジアム           |ベアスタ|鳥栖  |鳥栖スタジアム(鳥栖)|
|佐賀県総合運動場陸上競技場           |佐賀   |鳥栖   ||
|トランスコスモススタジアム長崎       |トラスタ|長崎  |長崎県立総合運動公園陸上競技場(長崎県立)|
|NDソフトスタジアム山形              |ＮＤスタ|山形  |山形県総合運動公園陸上競技場(山形県)|
|ケーズデンキスタジアム水戸           |Ｋｓスタ|水戸  |水戸市立競技場(水戸)|
|笠松運動公園陸上競技場              |笠松|水戸|※ＫｓスタはJ1基準を満たさないため登録されているが基本使われない|
|栃木県グリーンスタジアム            |栃木グ|栃木  ||
|NACK5スタジアム大宮               |ＮＡＣＫ|大宮  |大宮公園サッカー場(大宮)|
|熊谷スポーツ文化公園陸上競技場      |熊谷陸|大宮   ||
|フクダ電子アリーナ                 |フクアリ|千葉  ||
|市原緑地運動公園臨海競技場          |市原|千葉     ||
|味の素フィールド西が丘             |味フィ西|東京Ｖ|西が丘サッカー場(西が丘)|
|駒沢オリンピック公園総合運動場陸上競技場|駒沢|東京Ｖ||
|町田市立陸上競技場                 |町田|町田  ||
|ニッパツ三ツ沢球場                  |ニッパツ|横浜FC|三ツ沢公園球技場(三ッ沢)、別称として「ニッパ球」とも|
|三ツ沢公園陸上競技場               |三ツ沢陸|横浜FC|※三ッ沢公園内にある陸上競技場でニッパツとは別|
|山梨中銀スタジアム                  |中銀スタ|甲府  |山梨県小瀬スポーツ公園陸上競技場(小瀬)|
|松本平広域公園総合球技場             |松本|松本  |※「アルウィン」とも呼ばれるがデータには含まれず|
|デンカビッグスワンスタジアム         |デンカＳ|新潟  |新潟スタジアム(新潟ス)→東北電力ビッグスワンスタジアム(東北電ス)|
|石川県西部緑地公園陸上競技場         |石川西部|金沢  ||
|岐阜メモリアルセンター長良川競技場    |長良川|岐阜  ||
|京都市西京極総合運動公園陸上競技場兼球技場|西京極|京都  ||
|シティライトスタジアム              |Ｃスタ|岡山  |岡山県総合グラウンド陸上競技場(岡山)→カンコースタジアム(カンスタ)|
|岡山県津山陸上競技場                |津山|岡山    ||
|維新みらいふスタジアム              |維新公園|山口  ||
|下関市営下関陸上競技場              |下関|山口||
|Pikaraスタジアム                  |ピカスタ|讃岐  |香川県立丸亀競技場(丸亀)|
|鳴門・大塚スポーツパークポカリスエットスタジアム|鳴門大塚|徳島  |徳島県鳴門総合運動公園陸上競技場(鳴門)|
|ニンジニアスタジアム                |ニンスタ|愛媛  |愛媛県総合運動公園陸上競技場(愛媛陸)|
|レベルファイブスタジアム             |レベスタ|福岡  |東平尾公園博多の森球技場(博多球)|
|えがお健康スタジアム                 |えがおＳ|熊本  |熊本県民総合運動公園陸上競技場(熊本)→うまかな・よかなスタジアム(うまスタ)|
|熊本市水前寺競技場                   |水前寺|熊本||
|大分銀行ドーム                      |大銀ド|大分  |大分スポーツ公園総合競技場(大分)→九州石油ドーム(九石ド)、別称として「大分ス」|
|いわぎんスタジアム                   |いわスタ|盛岡  |盛岡南球技場(盛岡南)|
|北上総合運動公園陸上競技場         　　|北上   |盛岡| |
|あきぎんスタジアム                   |Ａ‐スタ|秋田  |秋田市八橋運動公園球技場(秋田球)|
|秋田市八橋運動公園陸上競技場           |秋田陸|秋田  ||
|とうほう・みんなのスタジアム           |とうスタ|福島  |福島県営あづま陸上競技場(福島)|
|あいづ陸上競技場                     |あいづ|福島||
|いわきグリーンフィールド              |いわきグ|福島||
|正田醤油スタジアム群馬                |正田スタ|群馬  |群馬県立敷島公園県営陸上競技場(群馬陸)|
|ニッパツ三ツ沢球技場                  |ニッパツ|YS横浜||
|相模原ギオンスタジアム                |ギオンス|相模原||
|長野Uスタジアム                      |長野Ｕ|長野  |南長野運動公園総合球技場(南長野)|
|富山県総合運動公園陸上競技場           |富山|富山  ||
|魚津桃山運動公園                      |魚津桃山|富山||
|藤枝総合運動公園サッカー場             |藤枝サ|藤枝  ||
|静岡県草薙総合運動場陸上競技場         |草薙陸|藤枝  ||
|愛鷹広域公園多目的競技場              |愛鷹|沼津  ||
|とりぎんバードスタジアム              |とりスタ|鳥取  ||
|チュウブYAJINスタジアム              |チュスタ|鳥取||
|ミクニワールドスタジアム北九州         |ミクスタ|北九州||
|白波スタジアム                      |鴨池|鹿児島||
|タピック県総ひやごんスタジアム         |沖縄県陸|琉球  ||
|沖縄市陸上競技場                      |沖縄市陸|琉球  ||

### 既に利用されなくなったスタジアム

|スタジアム                       |略称  |チーム|旧称|
|-------------------------------|-----|----|-----|
|国立競技場                       |国立 ||※改修工事のため2020年まで利用しない|
|入江運動公園陸上競技場             |室蘭|札幌|※2012年以降は開催されず|
|仙台市陸上競技場                  |宮城陸上|仙台|※以前仙台が利用|
|宮城県サッカー場                  |宮城サ|仙台|※以前仙台が利用|
|ひたちなか市総合運動公園           |ひたちな|鹿島|※以前鹿島が利用|
|川越運動公園                      |川越|浦和||
|江戸川区陸上競技場                |江戸川|FC東京||
|浜北平口サッカー場                 |浜松|磐田|1993年のみ|
|名古屋市瑞穂公園ラグビー場          |瑞穂球|名古屋|2007年ごろまで|
|三重県営鈴鹿スポーツガーデン        |鈴鹿|名古屋|2002年ごろまで|
|名古屋市港サッカー場               |名古屋港|名古屋|1994年まで|
|長居第2陸上競技場                  |長居２|Ｃ大阪|2007年ごろまで|
|東大阪市花園ラグビー場              |花園|Ｃ大阪|1999年のみ|
|皇子山陸上競技場                   |皇子山|Ｃ大阪|2002年のみ|
|兵庫県立三木総合防災公園陸上競技場   |三木陸|Ｃ大阪・神戸|2006-2007年のみ|
|広島県立びんご運動公園野球場        |びんご|広島|1993年のみ|
|福山市竹ヶ端運動公園陸上競技場      |福山市|広島|1994年のみ|
|長崎市総合運動公園                 |長崎市|長崎|2015年まで|
|鶴岡市小真木原陸上競技場            |鶴岡|山形|2001年まで|
|山形市陸上競技場                   |山形市|山形|2001年まで|
|日立市民運動公園陸上競技場          |日立市|水戸|2008年ごろまで|
|足利市総合運動公園陸上競技場        |足利陸|栃木|2009年ごろまで|
|鴻巣市上谷総合公園                 |鴻巣|大宮|2001年ごろまで|
|太田市運動公園陸上競技場           |太田陸|千葉|1992年のみ|
|韮崎中央公園陸上競技場             |韮崎中央|甲府|2000年まで|
|山梨県富士北麓公園陸上競技場        |富士北麓|甲府|2000年まで|
|新潟市陸上競技場                   |新潟陸|新潟|2003年まで、別称として「新潟」？|
|長良川球技メドウ                   |長良川球|岐阜|2007年まで|
|浅中公園陸上競技場                 |大垣|岐阜|2013年のみ|
|東平尾公園博多の森陸上競技場        |博多陸|福岡|2003年ごろまで|
|佐伯市総合運動公園陸上競技場        |佐伯|大分|2000年まで|
|大分市営陸上競技場                 |大分陸|大分|2002年まで|
|群馬県立敷島公園サッカー・ラグビー場 |群馬サ|群馬|2005年ごろまで|
|足利市総合運動公園陸上競技場        |足利陸|群馬|2009年ごろまで|
|長野市営陸上競技場                 |長野|長野|2014年ごろまで|
|佐久総合運動公園陸上競技場          |佐久|長野|2014年のみ|
|北九州市立本城陸上競技場              |本城|北九州|2016年ごろまで|

### 不明

* 札幌(札幌)→おそらく札幌ドと同じ
* 仙台(仙台)→恐らくユアスタと同じ
* 磐田(磐田)→おそらくヤマハと同じ
* 水戸(水戸)→おそらくＫｓスタと同じ
* 千葉(千葉)→フクアリと一緒かと思ったが1992年のみのデータであり、フクアリは2005年から。
* 新潟(新潟)→デンカＳと同じかと思ったがデータを見る限り新潟陸と一緒っぽい


In [10]:
len(df["スタジアム"].drop_duplicates())

173

In [24]:
df[df["ホーム"]=="秋田"]["スタジアム"].drop_duplicates()

11890    Ａ‐スタ
13332     秋田陸
Name: スタジアム, dtype: object

In [25]:
df[df["スタジアム"]=="本城"]

,年度,大会,節,試合日,K/O時刻,ホーム,スコア,アウェイ,スタジアム
3963,2002,Ｊ２,第３５節第１日,10/05(土),14:01,福岡,2-4,Ｃ大阪,本城
5084,2004,Ｊ２,第３６節第１日,09/26(日),18:04,福岡,1-3,京都,本城
8748,2010,Ｊ２,第２節第２日,03/14(日),13:05,北九州,1-3,徳島,本城
8766,2010,Ｊ２,第３節第２日,03/21(日・祝),16:01,北九州,1-0,東京Ｖ,本城
8806,2010,Ｊ２,第５節第２日,04/04(日),15:04,北九州,0-3,千葉,本城
8847,2010,Ｊ２,第７節第２日,04/18(日),13:03,北九州,1-1,栃木,本城
8878,2010,Ｊ２,第９節第１日,04/29(木・祝),19:03,北九州,0-1,岐阜,本城
8927,2010,Ｊ２,第１２節第２日,05/09(日),14:03,北九州,0-1,札幌,本城
8958,2010,Ｊ２,第１４節第１日,05/22(土),14:04,北九州,0-1,福岡,本城
8998,2010,Ｊ２,第１６節第２日,06/06(日),16:03,北九州,0-0,甲府,本城


In [11]:
def rename_studium(df, old, new):
    df["スタジアム"] = df["スタジアム"].replace(old,new)
    return df

df = rename_studium(df, "札幌", "札幌ド")
df = rename_studium(df, "仙台", "ユアスタ")
df = rename_studium(df, "東京", "味スタ")
df = rename_studium(df, "平塚", "ＢＭＷス")
df = rename_studium(df, "日本平", "アイスタ")
df = rename_studium(df, "アウスタ", "アイスタ")
df = rename_studium(df, "静岡", "エコパ")
df = rename_studium(df, "磐田", "ヤマハ")
df = rename_studium(df, "瑞穂陸", "パロ瑞穂")
df = rename_studium(df, "長居", "ヤンマー")
df = rename_studium(df, "神戸中央", "ノエスタ")
df = rename_studium(df, "神戸ウイ", "ノエスタ")
df = rename_studium(df, "ホムスタ", "ノエスタ")
df = rename_studium(df, "ホームズ", "ノエスタ")
df = rename_studium(df, "広島ビ", "Ｅスタ")
df = rename_studium(df, "鳥栖", "ベアスタ")
df = rename_studium(df, "長崎県立", "トラスタ")
df = rename_studium(df, "山形県", "ＮＤスタ")
df = rename_studium(df, "水戸", "Ｋｓスタ")
df = rename_studium(df, "大宮", "ＮＡＣＫ")
df = rename_studium(df, "西が丘", "味フィ西")
df = rename_studium(df, "三ツ沢", "ニッパツ")
df = rename_studium(df, "ニッパ球", "ニッパツ")
df = rename_studium(df, "小瀬", "ＮＤスタ")
df = rename_studium(df, "新潟ス", "デンカＳ")
df = rename_studium(df, "東北電ス", "デンカＳ")
df = rename_studium(df, "岡山", "Ｃスタ")
df = rename_studium(df, "カンスタ", "Ｃスタ")
df = rename_studium(df, "丸亀", "ピカスタ")
df = rename_studium(df, "鳴門", "鳴門大塚")
df = rename_studium(df, "愛媛陸", "ニンスタ")
df = rename_studium(df, "博多球", "レベスタ")
df = rename_studium(df, "熊本", "えがおＳ")
df = rename_studium(df, "うまスタ", "えがおＳ")
df = rename_studium(df, "大分", "大銀ド")
df = rename_studium(df, "九石ド", "大銀ド")
df = rename_studium(df, "大分ス", "大銀ド")
df = rename_studium(df, "盛岡南", "いわスタ")
df = rename_studium(df, "秋田球", "Ａ‐スタ")
df = rename_studium(df, "福島", "とうスタ")
df = rename_studium(df, "群馬陸", "正田スタ")
df = rename_studium(df, "南長野", "長野Ｕ")

In [12]:
len(df["スタジアム"].drop_duplicates())

131

## データベースを更新

修正済みのdfをデータベースに変換する。  
レコード単位でやると面倒なので、ファイルごと置き換える。

In [13]:
import os
os.remove(DB_NAME)
conn = sqlite3.connect(DB_NAME)
df.to_sql("result", conn)

# スクリプト化

donwload.py内にデータ加工(processing)としてまとめた。  
ダウンロード後、このノートブックの内容を実行せずに↓だけ実行すると、データ加工がおこなわれる。

In [1]:
import download
df = download.get_db_data()

In [2]:
len(df["スタジアム"].drop_duplicates())

354

In [3]:
import process_data
df = process_data.processing_data(df)

In [4]:
len(df["スタジアム"].drop_duplicates())

131

いったん↓を実行すると、
``` python
df = download.get_db_data()
```
のみで以降加工済みデータが得られるようになる。

In [5]:
process_data.db_update()

再度やると、「インターネット中継・TV放送」の行は無いというエラーになる。

In [6]:
df = download.get_db_data()
len(df["スタジアム"].drop_duplicates())

131